In [2]:
# Importações necessárias
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import pdfplumber
import re
import spacy
from nltk.corpus import stopwords
import chromadb
import nltk
nltk.download("stopwords")

# 1. Função para ler PDF
def ler_pdf(caminho_pdf):
    with pdfplumber.open(caminho_pdf) as leitor_pdf:
        texto = "".join([pagina.extract_text() for pagina in leitor_pdf.pages])
    return texto.replace("\n", " ")

# 2. Função de pré-processamento de texto
def tratamento_pln(texto):
    # Carregar modelo e stopwords
    nlp = spacy.load("pt_core_news_sm")
    stop_words = set(stopwords.words('portuguese'))
    
    # Normalização
    texto = texto.lower()
    texto = re.sub(r'[^a-zA-Záéíóú\s]', '', texto)
    
    # Tokenização e limpeza
    doc = nlp(texto)
    clean_tokens = [token.lemma_ for token in doc 
                   if token.text not in stop_words and not token.is_punct]
    
    return ' '.join(clean_tokens)

# 3. Divisão em chunks
def criar_chunks(texto, tamanho=40, overlap=10):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamanho,
        chunk_overlap=overlap
    )
    return splitter.split_text(texto)

# 4. Geração de embeddings e armazenamento
def criar_banco_vetorial(chunks, nome_colecao="colecao_teste"):
    # Gerar embeddings
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(chunks)
    
    # Criar banco vetorial
    client = chromadb.Client()
    collection = client.create_collection(name=nome_colecao)
    
    # Adicionar documentos
    ids = [f"doc_{i}" for i in range(len(chunks))]
    collection.add(
        documents=chunks,
        embeddings=embeddings.tolist(),  # Chroma espera listas
        ids=ids
    )
    
    return collection, model

# 5. Função de consulta
def consultar_banco(colecao, modelo, consulta, n_resultados=1):
    embedding_consulta = modelo.encode([consulta])
    resultados = colecao.query(
        query_embeddings=embedding_consulta.tolist(),
        n_results=n_resultados
    )
    return resultados

# Fluxo principal
if __name__ == "__main__":
    # Passo 1: Extrair texto
    texto = ler_pdf("chapeuzinho.pdf")
    
    # Passo 2: Pré-processar
    texto_tratado = tratamento_pln(texto)
    
    # Passo 3: Criar chunks
    chunks = criar_chunks(texto_tratado)
    
    # Passo 4: Banco vetorial
    colecao, modelo = criar_banco_vetorial(chunks, "contos_infantis")
    
    # Passo 5: Consulta de exemplo
    resultados = consultar_banco(colecao, modelo, "lobo mentiroso")
    
    # Exibir resultados
    for i in range(len(resultados['ids'][0])):
        print(f"ID: {resultados['ids'][0][i]}")
        print(f"Documento: {resultados['documents'][0][i]}")
        print(f"Distância: {resultados['distances'][0][i]}")
        print("-" * 40)

[nltk_data] Downloading package stopwords to /home/mateus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


InternalError: Collection [contos_infantis] already exists